In [1]:
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
np.random.seed(8)
import os
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 7.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Mounted at /content/drive


In [2]:
df_machineID = pd.read_csv("/content/drive/MyDrive/aitomatic/machineID_origin.csv",index_col=0)
df_age = pd.read_csv("/content/drive/MyDrive/aitomatic/age_origin.csv",index_col=0)
df_model = pd.read_csv("/content/drive/MyDrive/aitomatic/model_origin.csv",index_col=0)

In [3]:
train = pd.read_csv("/content/drive/MyDrive/aitomatic/train_origin.csv")
test = pd.read_csv("/content/drive/MyDrive/aitomatic/test_origin.csv")

In [4]:
train.drop(columns=['datetime'],inplace=True)
test.drop(columns=['datetime'],inplace=True)
# train = pd.get_dummies(train)
# test = pd.get_dummies(test)
train.head()

,machineID,volt,rotate,pressure,vibration,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5,model,age,failure
0,1,176.217853,418.504078,113.077935,45.087686,0.0,0.0,0.0,0.0,0.0,model3,18,0
1,1,162.879223,402.747490,95.460525,43.413973,0.0,0.0,0.0,0.0,0.0,model3,18,0
2,1,170.989902,527.349825,75.237905,34.178847,0.0,0.0,0.0,0.0,0.0,model3,18,0
3,1,162.462833,346.149335,109.248561,41.122144,0.0,0.0,0.0,0.0,0.0,model3,18,0
4,1,157.610021,435.376873,111.886648,25.990511,0.0,0.0,0.0,0.0,0.0,model3,18,0


In [33]:
#split train test
num_classes = 2
input_length = 24
n_upsampling = 12
hours = np.arange(input_length)
Q1, Q2, Q3 = input_length // 4, input_length // 2, input_length*3//4
x_train,y_train = [], []
#generate train
numerical_cols = ['volt', 'rotate', 'pressure', 'vibration']
mean_cols = ['mean_volt', 'mean_rotate', 'mean_pressure', 'mean_vibration']
std_cols = ['std_volt', 'std_rotate', 'std_pressure', 'std_vibration']
rank_cols = ['rank_volt', 'rank_rotate', 'rank_pressure', 'rank_vibration']
# columns = ['machineID','model', 'age'] + numerical_cols + ['anomaly']
columns = list(np.arange(12)) + numerical_cols + ['machineID','failure']
for group_name, df_group in train.groupby("machineID"):
  #normalize by machineID
  print("group_name",group_name)
  age  = df_group.iloc[0]['age']
  mdel = df_group.iloc[0]['model']
  df_group[np.arange(4)] = (df_group[numerical_cols].values - df_machineID.loc[group_name,mean_cols].values) / df_machineID.loc[group_name,std_cols].values
  df_group[np.arange(4,8)] = (df_group[numerical_cols].values - df_model.loc[mdel,mean_cols].values) / df_model.loc[mdel,std_cols].values
  df_group[np.arange(8,12)] = (df_group[numerical_cols].values - df_age.loc[age,mean_cols].values) / df_age.loc[age,std_cols].values
  df_group['machineID'] = df_machineID.loc[group_name,'machineID_rate']
  df_group = df_group[columns]
  # df_group['machineID'] = df_machine.loc[group_name,'machine_rate']
  # df_group[delta_cols] = df_group[numerical_cols].diff()
  # df_group = df_group[columns].fillna(0)

  leng = len(df_group)
  start_ids = np.arange(0,leng-input_length+1,input_length)
  for id in start_ids:
    X = df_group.iloc[id:id+input_length,:-1]
    X[numerical_cols] = (X[numerical_cols] - X[numerical_cols].mean(0)) / X[numerical_cols].std(0)#).astype(int)
    X[rank_cols] = X[np.arange(4)].rank()
    y = df_group.iloc[id:id+input_length,-1]
    label = y.max()
    x_train.append(X.values.astype(np.float32))
    y_train.append(label)
    #generate more data
    if label > 0 and leng- id > input_length and id:
      pos_ids = hours[(y > 0).values]
      for sample_id in np.random.randint(pos_ids.min()-Q3,pos_ids.max()-Q1+1,n_upsampling):
        X = df_group.iloc[id+sample_id:id+sample_id+input_length,:-1]
        X[numerical_cols] = (X[numerical_cols] - X[numerical_cols].mean(0)) / X[numerical_cols].std(0)#).astype(int)
        X[rank_cols] = X[np.arange(4)].rank()
        y = df_group.iloc[id+sample_id:id+sample_id+input_length,-1]
        x_train.append(X.values.astype(np.float32))
        y_train.append(y.max())
  start_ids +=  np.random.randint(Q1,Q3+1,len(start_ids))
  for rid in start_ids[:-1]:
    y = df_group.iloc[rid:rid+input_length,-1].max()
    if y > 0:
      continue
    X = df_group.iloc[rid:rid+input_length,:-1]
    X[numerical_cols] = (X[numerical_cols] - X[numerical_cols].mean(0)) / X[numerical_cols].std(0)#).astype(int)
    X[rank_cols] = X[np.arange(4)].rank()
    x_train.append(X.values.astype(np.float32))
    y_train.append(y)
x_train = np.array(x_train)
y_train_ = np.array(y_train)
y_train = np.zeros((y_train_.size, num_classes))
y_train[np.arange(y_train_.size), y_train_] = 1


group_name 1
group_name 2
group_name 3
group_name 4
group_name 5
group_name 6
group_name 7
group_name 8
group_name 9
group_name 10
group_name 11
group_name 12
group_name 13
group_name 14
group_name 15
group_name 16
group_name 17
group_name 18
group_name 19
group_name 20
group_name 21
group_name 22
group_name 23
group_name 24
group_name 25
group_name 26
group_name 27
group_name 28
group_name 29
group_name 30
group_name 31
group_name 32
group_name 33
group_name 34
group_name 35
group_name 36
group_name 37
group_name 38
group_name 39
group_name 40
group_name 41
group_name 42
group_name 43
group_name 44
group_name 45
group_name 46
group_name 47
group_name 48
group_name 49
group_name 50
group_name 51
group_name 52
group_name 53
group_name 54
group_name 55
group_name 56
group_name 57
group_name 58
group_name 59
group_name 60
group_name 61
group_name 62
group_name 63
group_name 64
group_name 65
group_name 66
group_name 67
group_name 68
group_name 69
group_name 70
group_name 71
group_name 72
g

In [34]:
#generate test
x_test,y_test = [], []
for group_name, df_group in test.groupby("machineID"):
  #normalize by machineID
  age  = df_group.iloc[0]['age']
  mdel = df_group.iloc[0]['model']
  df_group[np.arange(4)] = (df_group[numerical_cols].values - df_machineID.loc[group_name,mean_cols].values) / df_machineID.loc[group_name,std_cols].values
  df_group[np.arange(4,8)] = (df_group[numerical_cols].values - df_model.loc[mdel,mean_cols].values) / df_model.loc[mdel,std_cols].values
  df_group[np.arange(8,12)] = (df_group[numerical_cols].values - df_age.loc[age,mean_cols].values) / df_age.loc[age,std_cols].values
  df_group['machineID'] = df_machineID.loc[group_name,'machineID_rate']
  df_group = df_group[columns]
  leng = len(df_group)
  start_ids = np.arange(0,leng-input_length+1,input_length)
  start_ids = list(start_ids) + list(np.arange(input_length//3,leng-input_length+1,input_length)) + list(np.arange(2*input_length//3,leng-input_length+1,input_length))
  for id in start_ids:
    X = df_group.iloc[id:id+input_length,:-1]
    X[numerical_cols] = (X[numerical_cols] - X[numerical_cols].mean(0)) / X[numerical_cols].std(0)#).astype(int)
    X[rank_cols] = X[np.arange(4)].rank()
    y = df_group.iloc[id:id+input_length,-1].max()
    x_test.append(X.values.astype(np.float32))
    y_test.append(y)
x_test = np.array(x_test)
y_test_ = np.array(y_test)
y_test = np.zeros((y_test_.size, num_classes))
y_test[np.arange(y_test_.size), y_test_] = 1

In [37]:
x_train[0].shape

(24, 21)

In [38]:
x_test[0].shape

(24, 21)

In [23]:
#show label distribution after upsampling train data
len(y_train),len(y_test), np.unique(y_train.argmax(1),return_counts=True), np.unique(y_test.argmax(1),return_counts=True)

(69994,
 13900,
 (array([0, 1]), array([62047,  7947])),
 (array([0, 1]), array([13631,   269])))

In [47]:
x_train1 = x_train[...,:16]
x_test1 = x_test[...,:16]

In [48]:
def create_model():
  inputs = layers.Input(shape=x_train1.shape[1:], name='input')
  # x1 = layers.LSTM(units=50,return_sequences=True)(inputs)
  # x1 = layers.Dropout(0.2)(x1)
  # x1 = layers.LSTM(units=50,return_sequences=False)(inputs)
  # x1 = layers.Dropout(0.2)(x1)
  # outputs = layers.Dense(num_classes, activation="softmax")(x1)
  x1 = layers.Bidirectional(layers.LSTM(units=50,return_sequences=False))(inputs)
  # x1 = layers.Dropout(0.2)(x1)
  x2 = layers.Bidirectional(layers.LSTM(units=50,return_sequences=False))(inputs)
  x2 = layers.Dropout(0.2)(x2)
  # x3 = layers.Bidirectional(layers.LSTM(units=50,return_sequences=False))(inputs)
  # x3 = layers.Dropout(0.2)(x3)
  outputs = layers.Dense(num_classes, activation="softmax")(x2)
  return keras.models.Model(inputs=inputs, outputs=outputs)
class SaveBestModel(keras.callbacks.Callback):
    def __init__(self, filepath):
        super(SaveBestModel, self).__init__()
        self.filepath = filepath
        self.best_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs['val_f1_score'][1]
        if val_acc > self.best_acc:
            self.best_acc = val_acc
            self.model.save_weights(self.filepath, overwrite=True)
            print(f"Model saved with validation f1: {val_acc:.4f}")

model = create_model()
checkpoint_path = "/content/drive/MyDrive/aitomatic/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
callbacks = [SaveBestModel(filepath=checkpoint_path)]
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(0.0001),
    metrics=[tfa.metrics.F1Score(num_classes=num_classes),keras.metrics.AUC()],
)
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 24, 16)]          0         
                                                                 
 bidirectional_19 (Bidirecti  (None, 100)              26800     
 onal)                                                           
                                                                 
 dropout_9 (Dropout)         (None, 100)               0         
                                                                 
 dense_9 (Dense)             (None, 2)                 202       
                                                                 
Total params: 27,002
Trainable params: 27,002
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(
    x_train1,
    y_train,
    validation_data= (x_test1,y_test),
    epochs=100,
    batch_size=256,
    callbacks=callbacks,
)

Epoch 1/100
274/274 [==============================] - 8s 16ms/step - loss: 0.4281 - f1_score: 0.5519 - auc_8: 0.9082 - val_loss: 0.1250 - val_f1_score: 0.5833 - val_auc_8: 0.9954
Epoch 2/100
274/274 [==============================] - 3s 9ms/step - loss: 0.2322 - f1_score: 0.6795 - auc_8: 0.9671 - val_loss: 0.1149 - val_f1_score: 0.6332 - val_auc_8: 0.9943
Epoch 3/100
274/274 [==============================] - 2s 8ms/step - loss: 0.2045 - f1_score: 0.7505 - auc_8: 0.9739 - val_loss: 0.1052 - val_f1_score: 0.6392 - val_auc_8: 0.9947
Epoch 4/100
274/274 [==============================] - 2s 8ms/step - loss: 0.1890 - f1_score: 0.7855 - auc_8: 0.9774 - val_loss: 0.0977 - val_f1_score: 0.6381 - val_auc_8: 0.9949
Epoch 5/100
274/274 [==============================] - 2s 8ms/step - loss: 0.1798 - f1_score: 0.8055 - auc_8: 0.9793 - val_loss: 0.0992 - val_f1_score: 0.6413 - val_auc_8: 0.9946
Epoch 6/100
274/274 [==============================] - 3s 11ms/step - loss: 0.1733 - f1_score: 0.8195 - 

In [51]:
# model = create_model()
model.load_weights(checkpoint_path)
metric = tfa.metrics.F1Score(num_classes=num_classes)#num_classes=2
metric.update_state(y_test, model.predict(x_test1))
result = metric.result()
f1 = result.numpy()
print(f1, f1.mean())

435/435 [==============================] - 2s 3ms/step
[0.9786189  0.36604187] 0.6723304


In [32]:
model.load_weights("/content/drive/MyDrive/aitomatic/training_3/cp.ckpt")
metric.update_state(y_test, model.predict(x_test))
result = metric.result()
f1 = result.numpy()
print(f1, f1.mean())

435/435 [==============================] - 1s 3ms/step
[0.97889733 0.35746604] 0.66818166
